In [1]:
import pandas as pd
import numpy as np
import pickle
# 온라인 행동정보 데이터 + 구매데이터 + 고객 데이터

file='C:/LPOINT/01online.csv'
file1='C:/LPOINT/02trans.csv'
file2='C:/LPOINT/03client.csv'

online=pd.read_csv(file)
trans=pd.read_csv(file1)
client=pd.read_csv(file2)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
online

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm
0,7809,1,8,5,A03,20190913,01:16,2571103,NaN,NaN,34.0,2663.0,DIRECT,NaN
1,7809,1,4,2,A03,20190913,01:14,2485909,NaN,NaN,34.0,2663.0,DIRECT,NaN
2,7809,1,11,5,A03,20190913,01:17,2646597,NaN,NaN,34.0,2663.0,DIRECT,NaN
3,7809,1,1,2,A03,20190913,00:46,788304,NaN,NaN,34.0,2663.0,DIRECT,NaN
4,7809,1,9,5,A03,20190913,01:17,2617609,NaN,NaN,34.0,2663.0,DIRECT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196357,31040,1,1,0,A02,20190815,03:47,13525,NaN,다이슨 에어랩,4.0,62.0,unknown,mobile_web
3196358,4129,1,2,0,A02,20190721,01:22,250099,NaN,여성메탈시계,3.0,250.0,unknown,mobile_web
3196359,4129,1,1,0,A02,20190721,01:18,1525,NaN,여성메탈시계,3.0,250.0,unknown,mobile_web
3196360,54403,3,1,0,A02,20190731,20:14,16905,NaN,비비고왕교자,3.0,68.0,unknown,mobile_app


In [3]:
trans['on_off']=trans['biz_unit'].str.get(i=0) # on/off 변수 생성
trans['b_unit']=trans['biz_unit'].str.get(i=2) # 업종유형 1,2,3 변수 생성
trans=trans.astype({'de_dt':'str'})
trans['de_dt'] = trans['de_dt'].str.get(i=5) # de_dt : 7,8,9월로 범주화
online=online.astype({'action_type':'str'}) #action_type을 범주형으로 변환

In [4]:
trans

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct,on_off,b_unit
0,21922,104999,1,A03,unknown,9,12:41,5990,1,A,3
1,21279,104907,4,A03,unknown,9,10:27,10900,1,A,3
2,39423,105124,11,A03,unknown,9,17:26,12900,1,A,3
3,18362,104010,1,A03,unknown,9,09:57,9900,1,A,3
4,39423,105124,13,A03,0565,9,17:26,2990,1,A,3
...,...,...,...,...,...,...,...,...,...,...,...
599956,35311,2839,3,B03,0339,9,14:09,3500,1,B,3
599957,35311,2839,4,B03,0339,9,14:09,3600,2,B,3
599958,35311,2839,5,B03,0339,9,14:09,3500,1,B,3
599959,35311,2839,2,B03,0339,9,14:09,4500,1,B,3


## 데이터셋 병합

In [5]:
#online+client
data=pd.merge(online, client, on='clnt_id')
data=data.replace('unknown',np.nan)
on_sort=data.sort_values(by=['clnt_id','sess_id','hit_seq'])
on_sort=on_sort.dropna(subset=['trans_id']) #구매 데이터만 추출

In [6]:
# +trans
clu=pd.merge(on_sort, trans, on=['clnt_id', 'trans_id'])
clu=clu.replace('unknown',np.nan)
clu=clu.dropna(subset=['pd_c']) # pd_c 없는것 삭제

In [7]:
clu.isnull().sum()

clnt_id                 0
sess_id                 0
hit_seq                 0
action_type             0
biz_unit_x              0
sess_dt                 0
hit_tm                  0
hit_pss_tm              0
trans_id                0
sech_kwd           123095
tot_pag_view_ct         0
tot_sess_hr_v         295
trfc_src            10611
dvc_ctg_nm          88162
clnt_gender             0
clnt_age                0
trans_seq               0
biz_unit_y              0
pd_c                    0
de_dt                   0
de_tm                   0
buy_am                  0
buy_ct                  0
on_off                  0
b_unit                  0
dtype: int64

In [8]:
clu.describe(include='all')

,clnt_id,sess_id,hit_seq,action_type,biz_unit_x,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,clnt_age,trans_seq,biz_unit_y,pd_c,de_dt,de_tm,buy_am,buy_ct,on_off,b_unit
count,123095.000000,123095.000000,123095.000000,123095,123095,1.230950e+05,123095,1.230950e+05,123095.000000,0.0,...,123095,123095.000000,123095,123095,123095,123095,1.230950e+05,123095.000000,123095,123095
unique,NaN,NaN,NaN,2,3,NaN,1403,NaN,NaN,NaN,...,6,NaN,3,1059,3,1397,NaN,NaN,1,3
top,NaN,NaN,NaN,6,A03,NaN,10:31,NaN,NaN,NaN,...,40,NaN,A03,0964,7,10:31,NaN,NaN,A,3
freq,NaN,NaN,NaN,120094,112325,NaN,335,NaN,NaN,NaN,...,59978,NaN,112325,3665,43722,364,NaN,NaN,123095,112325
mean,38019.416231,18.445201,36.277859,NaN,NaN,2.019081e+07,NaN,1.560442e+06,75435.698452,NaN,...,NaN,4.920175,NaN,NaN,NaN,NaN,8.269653e+05,1.327503,NaN,NaN
std,19984.558889,27.618051,27.499017,NaN,NaN,8.170791e+01,NaN,1.315816e+06,21353.066515,NaN,...,NaN,3.464816,NaN,NaN,NaN,NaN,2.850229e+08,2.391913,NaN,NaN
min,2.000000,1.000000,1.000000,NaN,NaN,2.019070e+07,NaN,0.000000e+00,38872.000000,NaN,...,NaN,1.000000,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,NaN
25%,21278.000000,3.000000,17.000000,NaN,NaN,2.019072e+07,NaN,6.359960e+05,57070.500000,NaN,...,NaN,2.000000,NaN,NaN,NaN,NaN,2.490000e+03,1.000000,NaN,NaN
50%,37922.000000,9.000000,31.000000,NaN,NaN,2.019081e+07,NaN,1.223940e+06,74871.000000,NaN,...,NaN,4.000000,NaN,NaN,NaN,NaN,3.990000e+03,1.000000,NaN,NaN
75%,55393.000000,22.000000,49.000000,NaN,NaN,2.019090e+07,NaN,2.113358e+06,93454.000000,NaN,...,NaN,7.000000,NaN,NaN,NaN,NaN,7.980000e+03,1.000000,NaN,NaN


#### final_pd_c (clnt_sess당 pd_c 리스트 만들기)

In [9]:
df=clu[['clnt_id','sess_id','pd_c']]
#df['clnt_sess']=df['clnt_id']*1000+df['sess_id']
#사용자별 산 상품 갯수 출력하여 pd_c 리스트 반환
clnt_sess=df['clnt_id']
clnt_sess=list(clnt_sess)
count={}
for i in clnt_sess:
    try: count[i] += 1
    except: count[i]=1
print(count)

{2: 10, 9: 17, 23: 1, 24: 16, 38: 5, 40: 4, 45: 1, 49: 170, 54: 1, 57: 1, 60: 1, 64: 1, 80: 10, 92: 1, 95: 4, 106: 74, 124: 9, 135: 1, 137: 1, 151: 1, 183: 1, 202: 98, 211: 2, 214: 1, 218: 8, 226: 1, 228: 6, 269: 11, 271: 4, 275: 15, 277: 2, 282: 1, 285: 2, 290: 1, 317: 5, 321: 1, 333: 1, 336: 5, 353: 2, 376: 1, 385: 1, 420: 3, 422: 10, 434: 6, 456: 20, 475: 34, 493: 1, 507: 1, 521: 20, 536: 42, 538: 3, 548: 2, 551: 9, 585: 1, 592: 18, 624: 43, 625: 1, 627: 3, 635: 3, 642: 13, 665: 3, 686: 1, 690: 1, 695: 7, 696: 1, 706: 3, 714: 10, 747: 1, 817: 3, 853: 24, 870: 2, 880: 30, 887: 10, 908: 8, 909: 24, 924: 1, 946: 62, 948: 22, 955: 1, 958: 1, 959: 17, 969: 7, 982: 3, 983: 1, 991: 1, 1010: 1, 1014: 2, 1029: 1, 1057: 15, 1061: 1, 1073: 3, 1076: 1, 1082: 2, 1086: 1, 1111: 1, 1112: 5, 1119: 2, 1127: 1, 1130: 6, 1140: 43, 1149: 12, 1160: 9, 1166: 5, 1176: 55, 1181: 1, 1200: 1, 1210: 6, 1211: 6, 1217: 1, 1220: 2, 1230: 1, 1245: 8, 1251: 3, 1261: 5, 1267: 1, 1279: 1, 1283: 4, 1284: 2, 1292: 1, 

In [10]:
values=count.values()
values=list(values)

In [11]:
li=[0]
for a in range(len(values)):
    b=li[-1]+values[a]
    li.append(b)

In [12]:
pd_c=list(df['pd_c'])

In [13]:
# 사용자별 상품을 리스트로 묶기
pd=[]
for a in range(len(li)):
    if a+1<len(li):
        b=pd_c[li[a]:li[a+1]]
        pd.append(b)

In [14]:
key=list(count.keys())
len(key)

8851

In [15]:
clntsess_pd={}
for a in range(len(pd)):
    clnt_sess=key[a]
    pd_c=pd[a]
    clntsess_pd[clnt_sess]=pd_c

In [16]:
import pandas as pd
t=pd.DataFrame(list(clntsess_pd.items()), columns=['clnt_id','pd_c'])
t

,clnt_id,pd_c
0,2,"[0092, 0092, 0151, 0351, 0189, 0186, 0092, 139..."
1,9,"[0515, 1388, 1183, 0347, 0114, 0517, 1075, 124..."
2,23,[0838]
3,24,"[0348, 0517, 1245, 0155, 1398, 1401, 1388, 010..."
4,38,"[0585, 0947, 0952, 0934, 1437]"
...,...,...
8846,72373,"[0735, 0219, 0326, 0334, 0150, 1019, 1068, 0870]"
8847,72400,"[1017, 1017, 1064]"
8848,72410,[0350]
8849,72423,[0169]


In [17]:
clu=clu[['clnt_id','sess_id','b_unit','action_type','de_dt','de_tm','hit_pss_tm','tot_pag_view_ct','tot_sess_hr_v','trfc_src','dvc_ctg_nm','clnt_gender','clnt_age','buy_am', 'buy_ct']]

In [18]:
#de_tm전처리
clu['de_tm']=clu['de_tm'].str.extract('([0-99]+)\:')
clu=clu.astype({'de_tm':int})

In [19]:
#dvc_ctg_nm전처리
dvc_mapping={"mobile_web":1 , "mobile_app":2 , "PC":3}
clu['dvc_ctg_nm']=clu['dvc_ctg_nm'].map(dvc_mapping)

In [20]:
#trfc_src전처리
trfc_mapping={'DIRECT':1,'PUSH':2,'WEBSITE':3,'PORTAL_1':4,'PORTAL_2':5,'PORTAL_3':6 }
clu['trfc_src']=clu['trfc_src'].map(trfc_mapping)
clu

,clnt_id,sess_id,b_unit,action_type,de_dt,de_tm,hit_pss_tm,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm,clnt_gender,clnt_age,buy_am,buy_ct
0,2,1,3,6,7,23,791905,55.0,872.0,1.0,NaN,F,30,36000,20
1,2,8,3,6,7,21,3251794,85.0,3256.0,1.0,NaN,F,30,1800,1
2,2,8,3,6,7,21,3251794,85.0,3256.0,1.0,NaN,F,30,3990,1
3,2,8,3,6,7,21,3251794,85.0,3256.0,1.0,NaN,F,30,4690,1
4,2,8,3,6,7,21,3251794,85.0,3256.0,1.0,NaN,F,30,4290,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127353,72400,7,2,6,7,9,103042,9.0,103.0,NaN,2.0,F,40,78900,1
127354,72410,3,2,6,7,13,437775,19.0,961.0,NaN,3.0,F,40,14470,1
127355,72423,1,1,6,9,11,116283,24.0,114.0,NaN,2.0,F,30,479700,1
127356,72424,2,3,6,9,17,2410463,82.0,2445.0,4.0,3.0,M,40,19900,1


In [21]:
#clnt_gender 전처리
clnt_gender_mapping={'F':1,'M':2}
clu['clnt_gender']=clu['clnt_gender'].map(clnt_gender_mapping)

In [22]:
#tot_sess_hr_v전처리 (초단위)
# /60 = 분
clu['tot_sess_hr_v']=clu['tot_sess_hr_v']/60
clu

,clnt_id,sess_id,b_unit,action_type,de_dt,de_tm,hit_pss_tm,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm,clnt_gender,clnt_age,buy_am,buy_ct
0,2,1,3,6,7,23,791905,55.0,14.533333,1.0,NaN,1,30,36000,20
1,2,8,3,6,7,21,3251794,85.0,54.266667,1.0,NaN,1,30,1800,1
2,2,8,3,6,7,21,3251794,85.0,54.266667,1.0,NaN,1,30,3990,1
3,2,8,3,6,7,21,3251794,85.0,54.266667,1.0,NaN,1,30,4690,1
4,2,8,3,6,7,21,3251794,85.0,54.266667,1.0,NaN,1,30,4290,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127353,72400,7,2,6,7,9,103042,9.0,1.716667,NaN,2.0,1,40,78900,1
127354,72410,3,2,6,7,13,437775,19.0,16.016667,NaN,3.0,1,40,14470,1
127355,72423,1,1,6,9,11,116283,24.0,1.900000,NaN,2.0,1,30,479700,1
127356,72424,2,3,6,9,17,2410463,82.0,40.750000,4.0,3.0,2,40,19900,1


In [23]:
#hit_pss_tm 전처리
#밀리초 / 1000 = 초
#초 / 60 = 분
clu['hit_pss_tm']=clu['hit_pss_tm']/60000
clu

,clnt_id,sess_id,b_unit,action_type,de_dt,de_tm,hit_pss_tm,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm,clnt_gender,clnt_age,buy_am,buy_ct
0,2,1,3,6,7,23,13.198417,55.0,14.533333,1.0,NaN,1,30,36000,20
1,2,8,3,6,7,21,54.196567,85.0,54.266667,1.0,NaN,1,30,1800,1
2,2,8,3,6,7,21,54.196567,85.0,54.266667,1.0,NaN,1,30,3990,1
3,2,8,3,6,7,21,54.196567,85.0,54.266667,1.0,NaN,1,30,4690,1
4,2,8,3,6,7,21,54.196567,85.0,54.266667,1.0,NaN,1,30,4290,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127353,72400,7,2,6,7,9,1.717367,9.0,1.716667,NaN,2.0,1,40,78900,1
127354,72410,3,2,6,7,13,7.296250,19.0,16.016667,NaN,3.0,1,40,14470,1
127355,72423,1,1,6,9,11,1.938050,24.0,1.900000,NaN,2.0,1,30,479700,1
127356,72424,2,3,6,9,17,40.174383,82.0,40.750000,4.0,3.0,2,40,19900,1


In [24]:
#new_buy_am 전처리 = buy_am*buy_ct
clu['new_buy_am']=clu['buy_am']*clu['buy_ct']
clu=clu.drop('buy_am',axis=1,inplace=None)
clu=clu.drop('buy_ct',axis=1,inplace=None)

In [25]:
#인덱스 재설정
index=[x for x in range(len(clu))]
clu.index=index

In [26]:
count={}
for i in range(len(clu)):
    clnt_sess=clu.at[clu.index[i],'clnt_id']
    buy_am=clu.at[clu.index[i],'new_buy_am']
    try: count[clnt_sess] += buy_am
    except: count[clnt_sess]=buy_am
print(count)

{2: 801400, 9: 59930, 23: 27000, 24: 112890, 38: 104135, 40: 442500, 45: 37800, 49: 1084505, 54: 39900, 57: 41000, 60: 58900, 64: 65160, 80: 1019500, 92: 23900, 95: 65500, 106: 449670, 124: 37340, 135: 73950, 137: 84150, 151: 35000, 183: 18200, 202: 615562, 211: 64900, 214: 22500, 218: 59840, 226: 94000, 228: 41780, 269: 80173, 271: 32280, 275: 67620, 277: 33670, 282: 39900, 285: 288000, 290: 24750, 317: 51462, 321: 148000, 333: 125130, 336: 34490, 353: 233800, 376: 12500, 385: 9900, 420: 384000, 422: 100050, 434: 25810, 456: 295030, 475: 1720850, 493: 174000, 507: 80000, 521: 92300, 536: 140640, 538: 105050, 548: 294500, 551: 2342190, 585: 21000, 592: 222400, 624: 287860, 625: 40400, 627: 145000, 635: 32600, 642: 103130, 665: 468930, 686: 109000, 690: 94050, 695: 35650, 696: 803940, 706: 236700, 714: 83240, 747: 34310, 817: 144100, 853: 148098, 870: 80900, 880: 188170, 887: 52910, 908: 39104, 909: 1227960, 924: 53940, 946: 1334020, 948: 189940, 955: 348100, 958: 17110, 959: 204910, 96

In [27]:
#clu=clu.drop('clnt_id',axis=1,inplace=None)
clu=clu.drop('sess_id',axis=1,inplace=None)
clu=clu.astype('float')
clu=clu.groupby('clnt_id')
med_clu=clu.mean()

In [28]:
med_clu=med_clu.drop('new_buy_am',axis=1,inplace=None)
med_clu['tot_buy_am']=count.values()
med_clu.head()

,b_unit,action_type,de_dt,de_tm,hit_pss_tm,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm,clnt_gender,clnt_age,tot_buy_am
clnt_id,,,,,,,,,,,,
2.0,3.0,6.0,7.400000,21.600000,40.066648,59.000000,40.440000,1.0,NaN,1.0,30.0,801400
9.0,3.0,6.0,7.941176,16.764706,25.332960,38.588235,26.571569,1.0,NaN,1.0,40.0,59930
23.0,1.0,6.0,7.000000,0.000000,2.785817,107.000000,13.066667,NaN,2.0,1.0,40.0,27000
24.0,3.0,6.0,7.875000,9.000000,35.874350,68.125000,41.938542,1.0,1.0,1.0,40.0,112890
38.0,3.0,6.0,8.000000,16.000000,19.418300,30.000000,21.883333,2.0,1.0,1.0,40.0,104135


In [29]:
med_clu.describe(include='all')

,b_unit,action_type,de_dt,de_tm,hit_pss_tm,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm,clnt_gender,clnt_age,tot_buy_am
count,8851.000000,8851.000000,8851.000000,8851.000000,8851.000000,8851.000000,8845.000000,5305.000000,5503.000000,8851.000000,8851.000000,8.851000e+03
mean,2.508982,6.049751,7.904577,13.466479,20.131659,39.272338,23.551752,1.445633,1.632423,1.143487,37.555079,1.274730e+07
std,0.639918,0.168064,0.722880,5.214521,18.235264,33.036153,19.835344,0.820547,0.718806,0.350588,8.990214,1.068759e+09
min,1.000000,6.000000,7.000000,0.000000,0.000000,1.000000,0.066667,1.000000,1.000000,1.000000,10.000000,0.000000e+00
25%,2.000000,6.000000,7.000000,10.413501,6.810275,16.000000,9.250000,1.000000,1.000000,1.000000,30.000000,4.690000e+04
50%,3.000000,6.000000,8.000000,13.204082,15.118983,31.000000,18.597917,1.000000,1.000000,1.000000,40.000000,9.721000e+04
75%,3.000000,6.000000,8.421053,17.000000,28.061554,52.000000,32.038021,2.000000,2.000000,1.000000,40.000000,2.359650e+05
max,3.000000,7.000000,9.000000,23.000000,250.309067,347.000000,252.950000,6.000000,3.000000,2.000000,60.000000,1.000003e+11


In [30]:
cg=["b_unit","action_type","de_dt","de_tm","trfc_src","dvc_ctg_nm",'clnt_gender']
med_clu["b_unit"]=round(med_clu["b_unit"],0)
med_clu["action_type"]=round(med_clu["action_type"],0)
med_clu["de_dt"]=round(med_clu["de_dt"],0)
med_clu["de_tm"]=round(med_clu["de_tm"],0)
med_clu["trfc_src"]=round(med_clu["trfc_src"],0)
med_clu["dvc_ctg_nm"]=round(med_clu["dvc_ctg_nm"],0)
med_clu['clnt_gender']=round(med_clu['clnt_gender'],0)